# CS360 Machine Learning
## Assignment 9

1.

(a) Load IRIS dataset. Create a linearly separable dataset with two features of petal length and petal width and two classes 'versicolor' and 'virginica'.

(b) Perform a classification using SVM.

(c) Perform 5-fold cross-validation and report the class-wise and average accuracies.

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import confusion_matrix
from sklearn.datasets import load_iris
from sklearn.svm import LinearSVC
from math import sqrt
import pandas as pd
import numpy as np
import warnings

In [2]:
X, y = load_iris(return_X_y=True)
df = pd.DataFrame(X, columns=['sepal length', 'sepal width', 'petal length', 'petal width'])
df['cls'] = y

lin_df = df[df.cls != 0].drop(labels=['sepal length', 'sepal width'], axis=1)

y = lin_df['cls'].to_numpy()
y = [i-1 for i in y]
X = lin_df.drop('cls', axis=1).to_numpy()

In [3]:
# versicolor_accuracy, virginica_accuracy, overall_accuracy = list(), list(), list()

for i in range(5):
  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=True, random_state=i)
  model = LinearSVC(loss='hinge', random_state=21, max_iter=5000).fit(X_train, y_train)
  y_preds = model.predict(X_test)
  cm = confusion_matrix(y_test, y_preds)
  versi_acc = cm[0, 0] / (cm[0, 0] + cm[0, 1])
  virgi_acc = cm[1, 1] / (cm[1, 0] + cm[1, 1])
  overall_acc = model.score(X_test, y_test)
  print(f"{i+1}. Overall Accuracy: {overall_acc * 100: .2f}%\tClass Accuracy for Versicolor: {versi_acc * 100: .2f}%\tClass Accuracy for Virginica: {virgi_acc * 100: .2f}%")

1. Overall Accuracy:  100.00%	Class Accuracy for Versicolor:  100.00%	Class Accuracy for Virginica:  100.00%
2. Overall Accuracy:  85.00%	Class Accuracy for Versicolor:  100.00%	Class Accuracy for Virginica:  75.00%
3. Overall Accuracy:  80.00%	Class Accuracy for Versicolor:  92.31%	Class Accuracy for Virginica:  57.14%
4. Overall Accuracy:  95.00%	Class Accuracy for Versicolor:  100.00%	Class Accuracy for Virginica:  91.67%
5. Overall Accuracy:  95.00%	Class Accuracy for Versicolor:  92.86%	Class Accuracy for Virginica:  100.00%


2. Perform classification over IRIS dataset using radial basis function neural network. For the identification of the initial cluster centers, use the k-means algorithm. Report the individual class-wise accuracy, average accuracy and overall accuracy.

In [4]:
class RBF_NN():
  def __init__(self, feats, hidden_n, classes, prototypes, sigma=1, learning_rate=1):
    self._prototypes = prototypes
    assert prototypes.shape == (hidden_n, feats)
    self._feats = feats
    self._hidden_n = hidden_n
    if self._hidden_n < self._feats:
      warnings.warn('Hidden Layer in a RBF Network should have atleast as many neurons as in the input layer.')
    self._classes = classes
    self._weights = np.random.default_rng().standard_normal((hidden_n+1, classes))
    self._sigma2 = sigma ** 2
    self._lr = learning_rate

  def __add_bias(self, array):
    array = array.reshape(1, -1)
    return np.c_[array, np.ones(array.shape[0])]

  def __transform(self, x):
    # x = x.reshape(1, -1)
    hidden_activations = - np.linalg.norm(x - self._prototypes, axis=1) / (2 * self._sigma2)
    assert hidden_activations.shape == (self._hidden_n, )
    return self.__add_bias(np.exp(hidden_activations))

  def __output(self, activations):
    activations = activations.reshape(1, -1)
    # activations = self.__add_bias(activations)
    return activations @ self._weights

  def __threshold(self, y_hat, train=False):
    if train:
      y_hat[y_hat > 0] = 1
      y_hat[y_hat < 0] = 0
      return y_hat.reshape(1, -1)
    else:
      y_hat[y_hat != y_hat.max()] = 0
      y_hat[y_hat == y_hat.max()] = 1
      return y_hat.reshape(1, -1)

  def fit(self, X, y, epochs=10000):
    y = LabelBinarizer().fit_transform(y)
    for j in range(epochs):
      error = 0
      for i, x in enumerate(X):
        x = x.reshape(1, -1)
        h = self.__transform(x)
        y_hat = self.__threshold(self.__output(h), train=True)
        error += h.reshape(-1, 1) @ (y[i].reshape(1, -1) - y_hat)
      self._weights += self._lr * error
  
  def predict(self, X):
    y = np.zeros((X.shape[0], self._classes))
    for i, x in enumerate(X):
      x = x.reshape(1, -1)
      h = self.__transform(x)
      y_h = self.__threshold(self.__output(h))
      y[i, :] = y_h.reshape(1, -1)
    
    return y.argmax(axis=1)


def calc_dmax(prototypes):
  dmax = 0
  for i, center in enumerate(prototypes):
    for j in range(i+1, prototypes.shape[1]):
      d = np.linalg.norm(center - prototypes[j])
      # print(np.linalg.norm(center - prototypes[j]))
      if d > dmax:
        dmax = d
  return dmax

def accuracy(y_true, y_preds):
  correct = 0
  for i, y in enumerate(y_preds):
    if y == y_true[i]:
      correct += 1
  return correct / len(y_true)

# K-Means Implementation from Assignment 7

class kMeans():
  def __init__(self, dimension, k):
    self._dimension = dimension
    self._clusters = k
    self._prototypes = np.zeros((k, dimension))

  def __sample_with_prob(self, sample_space, probability, num_samples=1):
    idx = [i for i in range(sample_space.shape[0])]
    pos = np.random.choice(idx, p=probability)

    return sample_space[pos]

  def __initialize_prototypes(self, X):
    probs = None
    center = self.__sample_with_prob(X, probs)
    self._prototypes[0, :] = center

    for i in range(1, self._clusters):
      probs = self.__nearest_cluster_dist(X, i)
      probs = [j ** 2 for j in probs]
      tot = sum(probs)
      probs = [j/tot for j in probs]
      center = self.__sample_with_prob(X, probs)
      self._prototypes[i, :] = center

  def __nearest_cluster_dist(self, X, i=None, clust_assign=False):
    if i is not None:
      prototypes = self._prototypes[0:i, :]
    else:
      prototypes = self._prototypes
    dists = list()
    for x in X:
      temp = []
      for cluster in prototypes:
        temp.append(np.linalg.norm(x - cluster))
      if clust_assign:
        dists.append(np.argmin(temp))
      else:
        dists.append(np.min(temp))
    return dists
  
  def predict(self, X):
    return self.__nearest_cluster_dist(X, clust_assign=True)

  def __cluster_means(self, X, c):
    c = np.array(c)
    centroids = np.zeros(self._prototypes.shape)
    for k in range(self._clusters):
      cluster_group = X[c == k, :]
      centroids[k, :] = cluster_group.mean(axis=0)
    return centroids

  def __SSE(self, X, c):
    sse = 0
    for i, x in enumerate(X):
      sse += np.linalg.norm(x - self._prototypes[c[i]]) ** 2
    return sse

  def fit(self, X, max_iterations=3000):
    self.__initialize_prototypes(X)
    current = self._prototypes

    for i in range(max_iterations):
    # while True:
      c = self.__nearest_cluster_dist(X, clust_assign=True)
      centroids = self.__cluster_means(X, c)
      
      if np.array_equal(centroids, self._prototypes):
        # print(f"Converged in {i+1} iterations")
        return self.__SSE(X, c)

      self._prototypes = centroids
    
    print(f"Reached Max Iterations but not converged.")
    return self.__SSE(X, c)

In [5]:
X, y = load_iris(return_X_y=True)

cluster = kMeans(dimension=X.shape[1], k=10)
cluster.fit(X)
prototypes = cluster._prototypes

dmax = calc_dmax(prototypes)

sigma = lambda dmax, k: dmax / sqrt(2*k)

model = RBF_NN(feats=X.shape[1], hidden_n=10, classes=3, prototypes=prototypes, sigma=sigma(dmax, 10), learning_rate=1e-4)
model.fit(X, y)

y_preds = model.predict(X)

cm = confusion_matrix(y, y_preds)
acc_class1 = cm[0, 0] / cm[0, :].sum()
acc_class2 = cm[1, 1] / cm[1, :].sum()
acc_class3 = cm[2, 2] / cm[2, :].sum()

avg_acc = (acc_class1 + acc_class2 + acc_class3) / 3

acc = accuracy(y, y_preds)

output = f"""
  CLASS-WISE ACCURACY
  Setosa: {acc_class1 * 100: .2f}\tVersicolor: {acc_class2 * 100: .2f}\tVirginica: {acc_class3 * 100: .2f}

  Average Accuracy: {avg_acc * 100: .2f}
  Overall Accuracy: {acc * 100: .2f}
"""
print(output)


  CLASS-WISE ACCURACY
  Setosa:  100.00	Versicolor:  90.00	Virginica:  94.00

  Average Accuracy:  94.67
  Overall Accuracy:  94.67

